In [1]:
fib(n) = n < 2 ? n : fib(n-1) + fib(n-2)

fib (generic function with 1 method)

In [2]:
fib(10)

55

In [3]:
@time fib(30)

  0.004160 seconds


832040

In [4]:
using BenchmarkTools
@benchmark fib(30)

BenchmarkTools.Trial: 1291 samples with 1 evaluation.
 Range (min … max):  3.491 ms …   5.473 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.994 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.870 ms ± 257.846 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▄                          ▇ █▁                             
  ▄█▄▇█▃▃▃▄▄▃▄▃▃▃▃▂▂▃▃▂▃▂▃▂▂▅▅█▅██▄▄▃▃▃▃▂▃▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▁▂▂▂ ▃
  3.49 ms         Histogram: frequency by time        4.55 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [5]:
@time fib(40)

  0.470541 seconds


102334155

In [6]:
;cat fib.c

#include <stdio.h>

int fib(int n) {
	return n < 2 ? n : fib(n-1) + fib(n-2);
}

int main() {
	printf("Hello, C Fib\n");
    return 0;
}


In [7]:
;gcc -O2 -fPIC -shared fib.c -o libfib.dylib

In [8]:
Cint

Int32

In [9]:
cfib(n) = ccall((:fib, "libfib"), Cint, (Cint,), n)

cfib (generic function with 1 method)

In [10]:
@time cfib(10)

  0.000009 seconds


55

In [11]:
@time cfib(30)

  0.002805 seconds


832040

In [12]:
@benchmark cfib(30)

BenchmarkTools.Trial: 2197 samples with 1 evaluation.
 Range (min … max):  2.080 ms …   3.490 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.247 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.272 ms ± 177.891 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅ ▂▃▁   ▄▃▂▁           █▃▁▁▁▁▁▁                             
  ██▆███▅▇▆████▇█▇█▇▇▅▆▆▆▇███████████▆█▆▇█▇▆▇▆▆▇▇▆▆▅▅▆▄▅▄▄▃▄▄ █
  2.08 ms      Histogram: log(frequency) by time      2.76 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [13]:
@time cfib(40)

  0.300954 seconds


102334155

In [14]:
@time fib(40)

  0.477215 seconds


102334155

In [15]:
function addone!(x::Vector)
    for i in 1:length(x)
        x[i] += 1
    end
end

addone! (generic function with 1 method)

In [16]:
x = ones(Int, 1000000);
@time addone!(x)

  0.007113 seconds (14.73 k allocations: 850.331 KiB, 90.52% compilation time)


In [17]:
;cat addone.c

#include <stdio.h>

void addone32(int * x, int n) {
	for (int i=0; i < n; i++) {
		x[i] += 1;
	}
}

void addone64(long * x, long n) {
	for (int i=0; i < n; i++) {
		x[i] += 1;
	}
}


int main() {
	printf("Hello, C addone\n");
    return 0;
}


In [18]:
;gcc -O2 -fPIC -shared addone.c -o libaddone.dylib

In [19]:
Cint, Clong

(Int32, Int64)

In [20]:
caddone!(x::Vector{Int32}) = ccall((:addone32, "libaddone"), Cvoid, (Ptr{Cint}, Cint), x, length(x))
caddone!(x::Vector{Int64}) = ccall((:addone64, "libaddone"), Cvoid, (Ptr{Clong}, Clong), x, length(x))
caddone!(ones(Cint, 3))
caddone!(ones(Clong, 3))

In [ ]:
n = 10000
@btime addone!(x) setup=(x=ones(Cint, n)) 
@btime caddone!(x) setup=(x=ones(Cint, n)) 

In [ ]:
@btime addone!(x) setup=(x=ones(Clong, n)) 
@btime caddone!(x) setup=(x=ones(Clong, n)) 

In [ ]:
function addone_inbounds!(x::Vector)
    @inbounds for i in 1:length(x)
        x[i] += 1
    end
end
addone_inbounds!(ones(Cint, 3))
addone_inbounds!(ones(Clong, 3))

In [ ]:
@btime addone_inbounds!(x) setup=(x=ones(Cint, n))
@btime addone_inbounds!(x) setup=(x=ones(Clong, n))

In [ ]:
k = 100
A = ones(k, k)
B = ones(k, k)
@time A * B;

In [ ]:
@btime A * B;

In [ ]:
using LinearAlgebra 
k = 10000
A = ones(k)
B = ones(k)
@time dot(A, B)

In [ ]:
@btime dot(A, B)